In [7]:
!pip install open3d
!pip install trimesh
!pip install numpy-stl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
import re
temp_index =[]
temp_fail=[]


id =[]
RS_faces = []
RS_edges = []
RS_free_edges = []
RS_vertices = []
RS_genus = []
ASES_fac = []
ASES_edg = []
ASES_s_edg = []
ASES_vert = []
ASES_s_vert = []
ASES_R_h = []
ASES_C_h = []
ASES_genus = []
ASA_reent = []
ASA_toric = []
ASA_contact = []
ASA_SES = []
ASA_SAS = []
T_vetices = []
T_edges = []
T_faces = []
T_genus = []
T_density = []
NVA_SES_volume = []
NVA_SES_area = []

f= open("/content/drive/MyDrive/ML/true_negative_xyzr/cellulose_AA16_xyzr/nohup.out")
fh =f.readlines()

for i in range(0,len(fh)):
  x = re.search("^REDUCED SURFACE ...",fh[i])
  if x :
    temp_index.append(i)
  y = re.search("^Failed",fh[i])
  if y :
    temp_fail.append(i)

for i in range(0,len(temp_fail)):
  for j in range(0,len(temp_index)):
    if temp_index[j]>temp_fail[i]:
      for k in range(temp_index[j-1],temp_fail[i]):
        fh[k] = " "
      break

for i in range(0,len(fh)):
  x = re.search("^INPUT",fh[i])
  if x :
    id.append(fh[i].split()[1].split(".")[0])
  y = re.search("^    RS component  #faces  #edges #free_edges  #vertices   genus",fh[i])
  if y :
    temp_rs = fh[i+1].split()
    RS_faces.append(float(temp_rs[1]))
    RS_edges.append(float(temp_rs[2]))
    RS_free_edges.append(float(temp_rs[3]))
    RS_vertices.append(float(temp_rs[4]))
    RS_genus.append(float(temp_rs[5]))
  z = re.search("^    SES comp.  #fac.  #edg. #s_edg. #vert. #s_vert. R_h C_h genus",fh[i])
  if z:
    temp = fh[i+1].split()
    ASES_fac.append(float(temp[1]))
    ASES_edg.append(float(temp[2]))
    ASES_s_edg.append(float(temp[3]))
    ASES_vert.append(float(temp[4]))
    ASES_s_vert.append(float(temp[5]))
    ASES_R_h.append(float(temp[6]))
    ASES_C_h.append(float(temp[7]))
    ASES_genus.append(float(temp[8]))
  r = re.search("^    Comp. probe_radius,   reent,    toric,   contact    SES       SAS",fh[i])
  if r :
    temp = fh[i+1].split()
    ASA_reent.append(float(temp[2]))
    ASA_toric.append(float(temp[3]))
    ASA_contact.append(float(temp[4]))
    ASA_SES.append(float(temp[5]))
    ASA_SAS.append(float(temp[6]))
  t = re.search("^    component#, vertices,  edges,  faces, genus, density",fh[i])
  if t :
    temp = fh[i+1].split()
    T_vetices.append(float(temp[1]))
    T_edges.append(float(temp[2]))
    T_faces.append(float(temp[3]))
    T_genus.append(float(temp[4]))
    T_density.append(float(temp[5]))
  u = re.search("^    Comp. probe_radius SES_volume SES_area",fh[i])
  if u:
    temp = fh[i+1].split()
    NVA_SES_volume.append(float(temp[2]))
    NVA_SES_area.append(float(temp[3]))

In [ ]:
id [0]

'AAA53434_relaxed_rank_1_model_5'

In [ ]:
# for AA10 '5vg0','5vg1' '2lhs'
id.remove('5vg0')
id.remove('5vg1')
id.remove('2lhs')
id

In [14]:
len(T_density)

57

In [ ]:
len(T_density)

97

In [15]:
import open3d as o3d
import trimesh
import numpy as np
import re
from stl import mesh as ds
import os


Surface_Area = []
# Volume = []
Scale = []
Mesh_volume = []
Convex_volume = []
Concavity = []
Convexity = []
Compactness = []
Sphericity = []
Asphericity = []


for i in range(0,len(id)):
  txt = id[i]+".stl"
  stl_file="/content/drive/MyDrive/STL/cellulose_AA16_stl/"+txt

  your_mesh = ds.Mesh.from_file(stl_file)
  # Calculate the surface area of the mesh
  # First, get the vectors representing each triangle in the mesh
  vectors = your_mesh.vectors.reshape(-1, 3, 3)
  # Calculate the cross product of the vectors of each triangle
  cross_products = np.cross(vectors[:,1]-vectors[:,0], vectors[:,2]-vectors[:,0])
  # Calculate the area of each triangle by taking half of the magnitude of the cross product
  areas = 0.5*np.linalg.norm(cross_products, axis=1)
  # Sum up the areas of all the triangles to get the total surface area of the mesh
  surface_area = np.sum(areas)

  


  # Load the STL file into a Trimesh object

  mesh = trimesh.load(stl_file)

  # Calculate the convex hull of the mesh
  convex_hull = mesh.convex_hull

  # Calculate the volume of the mesh and the convex hull
  mesh_volume = mesh.volume
  volume = mesh.volume
  convex_hull_volume = convex_hull.volume
  surface = mesh.scale
  # Calculate the concavity and convexity of the mesh
  concavity = 1 - (mesh_volume / convex_hull_volume)
  convexity = 1 - concavity
  area = mesh.area

  # Calculate the compactness of the mesh
  compactness = (36 * 3.14159 * volume**2)**(1/3) / surface_area


  # Calculate the radius of a sphere with the same volume as the mesh
  radius = (3 * volume / (4 * 3.14159))**(1/3)

  # Calculate the surface area of a sphere with the same volume as the mesh
  sphere_surface_area = 4 * 3.14159 * radius**2
  sphericity = sphere_surface_area / surface_area


  print("scale:",surface)
  print("area:",area)
  print('mesh_volume: {:.2f}'.format(mesh_volume))
  print('convex_volume: {:.2f}'.format(convex_hull_volume))
  print('Concavity: {:.2f}%'.format(concavity * 100))
  print('Convexity: {:.2f}%'.format(convexity * 100))
  print('compactness:',compactness)
  print('Sphericity: {:.2f}'.format(sphericity))




  # Load the STL file into a mesh object

  mesh = o3d.io.read_triangle_mesh(stl_file)

  # Calculate the radius of the minimum bounding sphere of the mesh
  diameter = np.linalg.norm(mesh.get_max_bound() - mesh.get_min_bound())
  radius = diameter / 2

  # Calculate the moment of inertia of the mesh
  points = np.asarray(mesh.vertices)
  center_of_mass = np.mean(points, axis=0)
  points -= center_of_mass
  inertia_tensor = np.zeros((3, 3))
  for i in range(len(points)):
      inertia_tensor += np.outer(points[i], points[i])
  eigvals, _ = np.linalg.eig(inertia_tensor)

  # Calculate the asphericity of the mesh
  asphericity = max(eigvals) / radius**2 - 1

  print('Asphericity: {:.2f}'.format(asphericity))

  print("done:{}".format(txt))
  Surface_Area.append(surface_area)
  # Volume.append(volume)
  Scale.append(surface)
  Mesh_volume.append(mesh_volume)
  Convex_volume.append(convex_hull_volume)
  Concavity.append(concavity)
  Convexity.append(convexity)
  Compactness.append(compactness)
  Sphericity.append(sphericity)
  Asphericity.append(asphericity)

scale: 77.77902767106269
area: 6735.351693950021
mesh_volume: 17395.49
convex_volume: 34572.12
Concavity: 49.68%
Convexity: 50.32%
compactness: 0.4820383153759928
Sphericity: 0.48
Asphericity: 5558.01
done:AEO58740_relaxed_rank_1_model_4.stl
scale: 77.62257365362012
area: 6429.555528855295
mesh_volume: 21562.98
convex_volume: 35062.44
Concavity: 38.50%
Convexity: 61.50%
compactness: 0.5826961929128779
Sphericity: 0.58
Asphericity: 6067.50
done:AEO60424_relaxed_rank_1_model_4.stl
scale: 76.89615500329482
area: 6674.046181790573
mesh_volume: 22341.05
convex_volume: 37331.82
Concavity: 40.16%
Convexity: 59.84%
compactness: 0.5747740969303737
Sphericity: 0.57
Asphericity: 6747.64
done:AEO71000_relaxed_rank_1_model_5.stl
scale: 78.39866151141507
area: 6547.087206061001
mesh_volume: 22071.54
convex_volume: 35247.13
Concavity: 37.38%
Convexity: 62.62%
compactness: 0.5811982942462719
Sphericity: 0.58
Asphericity: 6253.06
done:APA09519_relaxed_rank_1_model_5.stl
scale: 73.81050312908131
area: 6

In [16]:
import pandas as pd

df = pd.DataFrame(list(zip(id,RS_faces,RS_edges,RS_free_edges,RS_vertices,RS_genus,
                           ASES_fac,ASES_edg,ASES_s_edg,ASES_vert,ASES_s_vert,
                           ASES_R_h,ASES_C_h,ASES_genus,ASA_reent,ASA_toric,
                           ASA_contact,ASA_SES,ASA_SAS,T_vetices,T_edges,
                           T_faces,T_genus,T_density,NVA_SES_volume,
                           NVA_SES_area,Surface_Area,Scale,Mesh_volume,
                           Convex_volume,Concavity,Convexity,Compactness,
                           Sphericity,Asphericity)),
                  columns = ["id","RS_faces","RS_edges","RS_free_edges","RS_vertices",
                             "RS_genus","ASES_fac","ASES_edg","ASES_s_edg",
                             "ASES_vert","ASES_s_vert","ASES_R_h","ASES_C_h",
                             "ASES_genus","ASA_reent","ASA_toric","ASA_contact",
                             "ASA_SES","ASA_SAS","T_vetices","T_edges","T_faces",
                             "T_genus","T_density","NVA_SES_volume","NVA_SES_area",
                             "Surface_Area","Scale","Mesh_volume","Convex_volume",
                             "Concavity","Convexity","Compactness","Sphericity",
                             "Asphericity" ])

In [17]:
df

,id,RS_faces,RS_edges,RS_free_edges,RS_vertices,RS_genus,ASES_fac,ASES_edg,ASES_s_edg,ASES_vert,...,NVA_SES_area,Surface_Area,Scale,Mesh_volume,Convex_volume,Concavity,Convexity,Compactness,Sphericity,Asphericity
0,AEO58740_relaxed_rank_1_model_4,2470.0,3705.0,3.0,1080.0,80.0,7613.0,15384.0,247.0,7772.0,...,6571.952,6735.351562,77.779028,17395.492971,34572.124878,0.496835,0.503165,0.482038,0.482038,5558.009682
1,AEO60424_relaxed_rank_1_model_4,2428.0,3642.0,1.0,1069.0,74.0,7462.0,15069.0,198.0,7610.0,...,6296.831,6429.555664,77.622574,21562.976445,35062.444032,0.385012,0.614988,0.582696,0.582696,6067.500828
2,AEO71000_relaxed_rank_1_model_5,2628.0,3942.0,1.0,1143.0,87.0,8045.0,16212.0,186.0,8170.0,...,6553.817,6674.046387,76.896155,22341.051493,37331.821944,0.401555,0.598445,0.574774,0.574774,6747.640281
3,APA09519_relaxed_rank_1_model_5,2468.0,3702.0,1.0,1086.0,75.5,7559.0,15228.0,163.0,7672.0,...,6429.301,6547.086914,78.398662,22071.540833,35247.130316,0.373806,0.626194,0.581198,0.581198,6253.064020
4,ATZ50322_relaxed_rank_1_model_5,2342.0,3513.0,0.0,1046.0,63.5,7133.0,14361.0,118.0,7230.0,...,6106.712,6250.080078,73.810503,21539.203714,33075.364039,0.348784,0.651216,0.598988,0.598988,6378.151670
5,BAE64879_relaxed_rank_1_model_3,2554.0,3831.0,0.0,1110.0,84.5,7841.0,15843.0,218.0,8004.0,...,6538.251,6714.692383,81.600726,22467.743887,36996.209127,0.392701,0.607299,0.573453,0.573453,5750.067392
6,BCR93324_relaxed_rank_1_model_5,2460.0,3690.0,0.0,1080.0,76.0,7520.0,15156.0,147.0,7638.0,...,6388.601,6563.038086,79.129515,21923.854001,35787.182933,0.387383,0.612617,0.577196,0.577196,5881.142666
7,BCS05968_relaxed_rank_1_model_5,2454.0,3681.0,0.0,1080.0,74.5,7499.0,15111.0,144.0,7614.0,...,6390.539,6568.512695,79.084616,21925.440712,35789.934553,0.387385,0.612615,0.576743,0.576743,5891.556093
8,BCS18392_relaxed_rank_1_model_3,2520.0,3780.0,1.0,1103.0,80.0,7718.0,15573.0,172.0,7858.0,...,6579.164,6764.755859,83.026014,22477.940950,37357.535373,0.398302,0.601698,0.569381,0.569381,5534.445699
9,BCS28198_relaxed_rank_1_model_1,2164.0,3246.0,0.0,941.0,71.5,6662.0,13470.0,202.0,6808.0,...,5664.369,5827.208008,73.293221,15624.596257,28834.617918,0.458131,0.541869,0.518676,0.518676,5827.853123


In [18]:
df.to_csv("raw_independent_set_AA16.csv")

In [ ]:
f_tn = open("/content/drive/MyDrive/ML/nw_tn.txt")
fv = f_tn.readlines()
for i in range(0,len(fv)):
  fv[i]=fv[i].strip()
fv.pop(len(fv)-1)

''

In [ ]:
id =[]
RS_faces = []
RS_edges = []
RS_free_edges = []
RS_vertices = []
RS_genus = []
ASES_fac = []
ASES_edg = []
ASES_s_edg = []
ASES_vert = []
ASES_s_vert = []
ASES_R_h = []
ASES_C_h = []
ASES_genus = []
ASA_reent = []
ASA_toric = []
ASA_contact = []
ASA_SES = []
ASA_SAS = []
T_vetices = []
T_edges = []
T_faces = []
T_genus = []
T_density = []
NVA_SES_volume = []
NVA_SES_area = []
Surface_Area = []
Scale = []
Mesh_volume = []
Convex_volume = []
Concavity = []
Convexity = []
Compactness = []
Sphericity = []
Asphericity = []

for i in range(0,len(df)):
  if df["id"][i].strip() in fv:
    id.append(df["id"][i])
    RS_faces.append(df["RS_faces"][i])
    RS_edges.append(df["RS_edges"][i])
    RS_free_edges.append(df["RS_free_edges"][i])
    RS_vertices.append(df["RS_vertices"][i])
    RS_genus.append(df["RS_genus"][i])
    ASES_fac.append(df["ASES_fac"][i])
    ASES_edg.append(df["ASES_edg"][i])
    ASES_s_edg.append(df["ASES_s_edg"][i])
    ASES_vert.append(df["ASES_vert"][i])
    ASES_s_vert.append(df["ASES_s_vert"][i])
    ASES_R_h.append(df["ASES_R_h"][i])
    ASES_C_h.append(df["ASES_C_h"][i])
    ASES_genus.append(df["ASES_genus"][i])
    ASA_reent.append(df["ASA_reent"][i])
    ASA_toric.append(df["ASA_toric"][i])
    ASA_contact.append(df["ASA_contact"][i])
    ASA_SES.append(df["ASA_SES"][i])
    ASA_SAS.append(df["ASA_SAS"][i])
    T_vetices.append(df["T_vetices"][i])
    T_edges.append(df["T_edges"][i])
    T_faces.append(df["T_faces"][i])
    T_genus.append(df["T_genus"][i])
    T_density.append(df["T_density"][i])
    NVA_SES_volume.append(df["NVA_SES_volume"][i])
    NVA_SES_area.append(df["NVA_SES_area"][i])
    Surface_Area.append(df["Surface_Area"][i])
    Scale.append(df["Scale"][i])
    Mesh_volume.append(df["Mesh_volume"][i])
    Convex_volume.append(df["Convex_volume"][i])
    Concavity.append(df["Concavity"][i])
    Convexity.append(df["Convexity"][i])
    Compactness.append(df["Compactness"][i])
    Sphericity.append(df["Sphericity"][i])
    Asphericity.append(df["Asphericity"][i])

In [ ]:
import pandas as pd

df = pd.DataFrame(list(zip(id,RS_faces,RS_edges,RS_free_edges,RS_vertices,RS_genus,
                           ASES_fac,ASES_edg,ASES_s_edg,ASES_vert,ASES_s_vert,
                           ASES_R_h,ASES_C_h,ASES_genus,ASA_reent,ASA_toric,
                           ASA_contact,ASA_SES,ASA_SAS,T_vetices,T_edges,
                           T_faces,T_genus,T_density,NVA_SES_volume,
                           NVA_SES_area,Surface_Area,Scale,Mesh_volume,
                           Convex_volume,Concavity,Convexity,Compactness,
                           Sphericity,Asphericity)),
                  columns = ["id","RS_faces","RS_edges","RS_free_edges","RS_vertices",
                             "RS_genus","ASES_fac","ASES_edg","ASES_s_edg",
                             "ASES_vert","ASES_s_vert","ASES_R_h","ASES_C_h",
                             "ASES_genus","ASA_reent","ASA_toric","ASA_contact",
                             "ASA_SES","ASA_SAS","T_vetices","T_edges","T_faces",
                             "T_genus","T_density","NVA_SES_volume","NVA_SES_area",
                             "Surface_Area","Scale","Mesh_volume","Convex_volume",
                             "Concavity","Convexity","Compactness","Sphericity",
                             "Asphericity" ])

In [ ]:
df.to_csv("true_negative_nw.csv")

In [ ]:
import re
id = []
f= open("/content/drive/MyDrive/ML/existing_struc/structures/AA10_stl/nohup.out")
fh =f.readlines()

for i in range(0,len(fh)):
  x = re.search("^INPUT",fh[i])
  if x :
    id.append(fh[i].split()[1].split(".")[0])

In [ ]:
import requests , sys 
import re
function = []
id  = ['4mai','4mah','7p3u','40pb','5lsv','5t7j','5t7k','5t7n','6tbq','6tbr','6tc4','5no7','5msz','6z5y']


for i in range(0,len(id)):
  txt = id[i]
  requesturls = "https://www.ebi.ac.uk/pdbe/api//mappings/uniprot/"+txt+"?pretty=true"
  r =requests.get(requesturls).json()
  a = str(r[txt.lower()]['UniProt'].keys())
  try:
    n = a.split("'")[1]
    requestURL = "https://www.ebi.ac.uk/proteins/api/proteins?offset=0&size=100&accession="+n
    r = requests.get(requestURL, headers={ "Accept" : "application/xml"})
    func = ""
    print(r.text)
    for i in r.text.split("<"):
      x = re.search("^property type=\"term\"",i.split("/>")[0])
      if x :
        func += i[28:-2]
    function.append(func)
  except:
    function.append("Cant_find_any_annotation")

In [ ]:
import pandas as pd

sf = pd.DataFrame(list(zip(id,function)),columns=["id","function"])
sf

In [ ]:
function

['F:metal ion binding"',
 'F:metal ion binding"',
 '',
 'Cant_find_any_annotation',
 'C:membrane"F:metal ion binding"',
 'C:membrane"F:metal ion binding"',
 'C:membrane"F:metal ion binding"',
 'C:membrane"F:metal ion binding"',
 'C:membrane"F:metal ion binding"',
 'C:membrane"F:metal ion binding"',
 'C:membrane"F:metal ion binding"',
 '',
 '',
 '']

In [ ]:
requestURL = "https://www.ebi.ac.uk/proteins/api/proteins?offset=0&size=100&accession=A0A0S2GKZ1"

r = requests.get(requestURL, headers={ "Accept" : "application/xml"})

In [ ]:
import re

for i in r.text.split("<"):
  x = re.search("^property type=\"term\"",i.split("/>")[0])
  if x :
    print(i[28:-3])

C:extracellular region
F:cellulase activity
F:cellulose binding
F:metal ion binding
P:cellulose catabolic process


In [ ]:
r.text

'<?xml version=\'1.0\' encoding=\'UTF-8\'?><uniprot xmlns="http://uniprot.org/uniprot" xsi:schemaLocation="http://uniprot.org/uniprot http://www.uniprot.org/support/docs/uniprot.xsd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"><entry xmlns="http://uniprot.org/uniprot" dataset="TrEMBL" created="2011-10-19" modified="2023-02-22" version="48"><accession>G0R6T8</accession><name>G0R6T8_HYPJQ</name><protein><submittedName><fullName evidence="10">Glycoside hydrolase family 61</fullName></submittedName></protein><gene><name evidence="10" type="primary">cel61a</name><name evidence="10" type="ORF">TRIREDRAFT_73643</name></gene><organism evidence="11"><name type="scientific">Hypocrea jecorina (strain QM6a)</name><name type="common">Trichoderma reesei</name><dbReference type="NCBI Taxonomy" id="431241"/><lineage><taxon>Eukaryota</taxon><taxon>Fungi</taxon><taxon>Dikarya</taxon><taxon>Ascomycota</taxon><taxon>Pezizomycotina</taxon><taxon>Sordariomycetes</taxon><taxon>Hypocreomycetidae</ta

In [ ]:
a ="property type=\"term\" value=\""
print(len(a))


28
